In [39]:
from matplotlib.pyplot import *
import numpy
from IPython.display import display

In [40]:
import praw

reddit = praw.Reddit(client_id='Bmkuwt-Cz8rTgw',
                     client_secret='M2WDVEEaXfQqAwKgibu6YSdh5Ec',
                     password='^*i%bHAi32Yk',
                     user_agent='testscript by /u/rkeb',
                     username='rkeb')

In [41]:
reddit.config.store_json_result = True
subreddit = reddit.subreddit("all")

## Topic Analysis of ex_addict_bro's comments

In [42]:
#Topics for each subreddit?
#Which topic did they comment on?

In [43]:
import pandas as pd
import datetime as dt

# COMMENTS FOR A USER
sub_comments_dict = { "body":[]}

for comment in reddit.redditor('ex_addict_bro').comments.top('all'):
    sub_comments_dict['body'].append(comment.body)
    
#sub_comments = [sub for sub in sub_comments if not sub.domain.startswith('self.')]

pd.set_option("display.max_colwidth", 200)
topics_data = pd.DataFrame(sub_comments_dict)

# removing everything except alphabets`
topics_data['body'] = topics_data['body'].replace("[^a-zA-Z#]", " ")
# removing short words
topics_data['body'] = topics_data['body'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# make all text lowercase
topics_data['body'] = topics_data['body'].apply(lambda x: x.lower())    

display(topics_data)

,body
0,think great.
1,"this rant mad!!!111 don't want lift don't want have muscles"" don't want look like man"" that's words swolemädchen haven been mislead broki, that hear most often! there whey your chalice, what going..."
2,"wants break contact with that other girl. when that, gone. similar situation happened life. this about you, this about her. ""take back"", means value, that other girl does not. noticed, that aren't..."
3,"making mistake. cheating addiction. showing that addiction mistake codependency. even worst addiction. focus actions: leave that bitch, don't live with never ever, divorce her. soon possible. your..."
4,wife classic redhead your wife classic reflection you.
5,"100% purple pill. problem didn't read article about being purple pill, i've read article about low-iq baby-faced hobbit wearing salvation army sweater built well enough, because that's what ~~girl..."
6,"this mainly those that know background ""read post history, please, because i've been busy posting shit instead reading fucking book from sidebar"" long story short wife just figured deleted some te..."
7,"instead looking excuses, started investigating solutions lift diet."
8,"block everywhere, phone, facebook, email. never talk again."
9,"reason lose frame over some guy, obviously either hangover looked someone pass anger whatever. some guards like show their authoritaeee respect that, always humble front them, shake their hands th..."


In [98]:
def lsa(pandas_series): #tokenize
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')

    # tokenization
    tokenized_doc = pandas_series.apply(lambda x: x.split())

    # remove stop-words
    tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

    # de-tokenization
    detokenized_doc = []
    for i in range(len(topics_data)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)

    sub_comments = detokenized_doc

    #Document-Term Matrix
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer(stop_words='english', 
                                 #max_features= 500, # keep top 1000 terms 
                                 max_df = 0.5,
                                 use_idf=True,
                                 smooth_idf=True)


    document_term_matrix = vectorizer.fit_transform(pandas_series)

    from sklearn.decomposition import TruncatedSVD

    # SVD represent documents and terms in vectors 
    n_topics = 3
    lsa_model = TruncatedSVD(n_components=n_topics, 
                             algorithm='randomized', 
                             n_iter=100, 
                             random_state=122)

    terms = vectorizer.get_feature_names()
    lsa_model.fit(document_term_matrix)
    return lsa_model

def get_topics(lsa_model):
    topics = {}
    for i, comp in enumerate(lsa_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic_num = "Topic " + str(i) + ": "
        topic_words = ""
        for t in sorted_terms:
            topic_words= topic_words + t[0] + " "
        topics[topic_num] = topic_words
    return topics

In [99]:
lsa_model = lsa(topics_data['body'])
display(get_topics(lsa_model))

{'Topic 0: ': 'freelance fresh line donut veganized tasty wedding ',
 'Topic 1: ': 'pizza stop wwf line woolies make hard ',
 'Topic 2: ': 'greens roommate line meme caught toy goals '}

## Topic Analysis for Vegan (313k), VeganCircleJerk (23k), AskReddit (221m), TodayILearned (20m), TheRedPill (235k), Atheism (2.4m)

In [100]:
import csv

y_vegan = []
y_vegancj = []
y_askreddit = []
y_trd = []
y_til = []
y_atheism = []

with open('submissions.csv') as sub_file:
    subreader = csv.reader(sub_file)
    for row in subreader:
        if row[2] == "vegan":
            y_vegan.append(row[0])
        elif row[2] == "vegancirclejerk":
            y_vegancj.append(row[0])
        elif row[2] == "AskReddit":
            y_askreddit.append(row[0])
        elif row[2] == "todayilearned":
            y_til.append(row[0])
        elif row[2] == "TheRedPill":
            y_trd.append(row[0])
        elif row[2] == "atheism":
            y_atheism.append(row[0])
            
topics_data['y_vegan'] = pd.DataFrame(y_vegan)
topics_data['y_vegancj'] = pd.DataFrame(y_vegancj)
topics_data['y_askreddit'] = pd.DataFrame(y_askreddit)
topics_data['y_trd'] = pd.DataFrame(y_trd)
topics_data['y_til'] = pd.DataFrame(y_til)
topics_data['y_atheism'] = pd.DataFrame(y_atheism)         

In [101]:
lsa_model = lsa(topics_data['y_vegan'])
display(get_topics(lsa_model))

{'Topic 0: ': 'eats celiac annoyed ed cooked common beechwood ',
 'Topic 1: ': 'celiac bakery cider anti beechwood beach denver ',
 'Topic 2: ': 'ed common ect 570 cupcakes attack community '}

In [102]:
lsa_model = lsa(topics_data['y_vegancj'])
display(get_topics(lsa_model))

{'Topic 0: ': 'coconut beloved beefy act condemn coca appetite ',
 'Topic 1: ': 'beer anniversary 110 alternatives bag beekeeping buttermilk ',
 'Topic 2: ': 'collection browned commercial checked annual attack baked '}

In [103]:
lsa_model = lsa(topics_data['y_askreddit'])
display(get_topics(lsa_model))

{'Topic 0: ': 'council baking dumb dyes definition companies digestive ',
 'Topic 1: ': 'billion biscuit beans crumbles amazing big church ',
 'Topic 2: ': 'baking celiac bio companies dairy biscuit disperse '}

In [104]:
lsa_model = lsa(topics_data['y_trd'])
display(get_topics(lsa_model))

{'Topic 0: ': 'concerns director blank daiya entire doesn alpaca ',
 'Topic 1: ': 'chili emotions bell billion cooking beans anyform ',
 'Topic 2: ': 'beans broken blood energy aussies entire chef '}

In [105]:
lsa_model = lsa(topics_data['y_til'])
display(get_topics(lsa_model))

{'Topic 0: ': 'happier isn essentials pacific project belgium problem ',
 'Topic 1: ': 'belgium dietary phl isn checked fist farfalle ',
 'Topic 2: ': 'lactose present feelsgoodman help assumed oprah basically '}

In [106]:
lsa_model = lsa(topics_data['y_atheism'])
display(get_topics(lsa_model))

{'Topic 0: ': 'drink adds argument easy continue canines cake ',
 'Topic 1: ': 'continue bomb clowns asking finally amazed biomarkers ',
 'Topic 2: ': 'announcing community articles craving footprint access farfalle '}